In [ ]:
##########1. 필요한 패키지 임포트
import tkinter as tk
from tkinter import *
import tkinter.filedialog
from PIL import ImageTk, Image
import numpy as np
from keras.models import load_model
import cv2
import numpy 
import glob
import librosa
import pygame
from pygame import mixer  # 소리 재생 모듈입니다.
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import os


##########2. 레이아웃 설정

#인터페이스 창
top=Tk() # 창 생성
top.title('human and AI sound Classification')
top.geometry("665x700+600+200") # 창 크기 및 화면에서의 위치
top.resizable(False, False) # 크기변경 막아둠

#1번 프레임 생성
frame1=tkinter.Frame(top, relief='groove',bd=2, background="#393939")
frame1.pack(side="left",fill='both',expand=True)

#2번 프레임 생성
frame2=tkinter.Frame(top,relief="groove",bd=2, width=265, background="#c7c7c7")
frame2.pack(side="right",fill="both",expand=True)

#로고 얹기
logo = PhotoImage(file="C:\\data\\logo(1).png")
logo_label = Label(top ,image=logo,width=170, height=170, bd=0)
logo_label.pack(expand=1)
logo_label.place(x=14, y=20)


##########3. 기능 구현

#1.
main_img = 'c:\\data\\main1.png'  # 초기 이미지 경로 설정★★★
test_num = 0     # 현재 테스트 파일 번호
is_off = True   # 소리 재생 여부
graph_flag = False  # 그래프 존재 여부

# 2. upload_sound 함수 생성 
def upload_sound():
    try:
        global test_list
        global test_num
        global x_test_list
        global wave_list
                
        test_list = filedialog.askopenfilenames()  # 윈도우 탐색기를 엽니다.
        x_test_list = [librosa.load(x)[0] for x in test_list]  # 숫자로 변환하고
        wave_list = [librosa.load(x)[1] for x in test_list]    # 주파수 대역을 추출
        test_num = 0
        
        Next('first')
        Draw_img(main_img)
        label_fname(0)
        
    except:
        pass  # try 와 except 사이에 문제가 생겨도 그냥 무시해라!


# 3. 모델을 불러옵니다.
model = load_model("C:\\data\\voice_model_final.h5")  # 모델 경로 설정★★★


# 4. 음성에 특징을 뽑아내는 함수 생성
# 특징은 소리의 주파수 스펙트럼 데이터 40개 + zero_crossing_rate 1개
def extract_features(audio_samples, sample_rate):  # 음성의 특징을 추출하는 함수 
    extracted_features = np.empty((0, 41, ))  # (1,41) 은 아니고 그냥 41개의 값을 받을 비어있는 리스트를 할당하겠다는 뜻
    
    if not isinstance(audio_samples, list):   # 리스트가 아니라면 
        audio_samples = [audio_samples]       # 리스트화 해라
        
    for sample in audio_samples:  # 진폭 데이터 리스트를 하나씩 가져옵니다. 
        zero_cross_feat = librosa.feature.zero_crossing_rate(sample).mean()  # 음성 신호 파형이 중심축(0) 을 통과하는 횟수
        mfccs = librosa.feature.mfcc(y=sample, sr=sample_rate, n_mfcc=40)  # https://youdaeng-com.tistory.com/5
        mfccsscaled = np.mean(mfccs.T,axis=0)   # 각 주파수별 평균값을 구합니다.
        mfccsscaled = np.append(mfccsscaled, zero_cross_feat)  # 주파수 40개에 대한 평균값 40개와 zero_cross_feat 값을 가지고
        mfccsscaled = mfccsscaled.reshape(1, 41, )  # 41개의 값을 학습 데이터로 구성합니다. [[293,291,293,...392]]
        extracted_features = np.vstack((extracted_features, mfccsscaled))  # [[293,291,293,...392]]
        
    return extracted_features


# 5. 위에서 출력되는 확률 백터 [[0.11749879 0.8825012]] 를 human 또는 AI 로 출력되게 하시오!
#  음성---> 모델 ---> 결과(cat 또는 dog)
def classify():
    if not test_list:  # 선택한 파일이 없다면
        tk.messagebox.showwarning("", "파일을 선택해주세요.")
        
    else:
        x_test_features = extract_features(x_test_list[test_num], wave_list[test_num])  # 41개의 학습 데이터를 만든 후에
        pred = model.predict(x_test_features.reshape(1,41,))   # 모델에 넣고 예측해라
        a = np.argmax(pred)  # 가장 큰 원소의 인덱스 번호를 추출합니다.

        if a == 0:
            sign = 'Real Human Voice'
            image_path = 'c:\\data\\real_voice_re.jpg'  # 사람 이미지 경로 설정★★★
        else:
            sign = 'Fake Human Voice'
            image_path = 'c:\\data\\fake_voice_re.jpg'  # AI 이미지 경로 설정★★★

        label.configure(background='#c7c7c7',foreground='#142f3a', text=sign, font=('arial',30, "bold"))  # 판에 자리잡은 영역에 글씨를 붙입니다.
        label.place(x=250, y=160)

        Draw_img(image_path)
        Draw_graph(pred)  # 분류 버튼을 누르면 그래프가 그려지도록 여기에 둡니다.

        
#6. 현재 재생중인 파일명 가져오는 함수
def label_fname(test_num):
    global test_list
    
    if test_list:
        file_name = os.path.splitext(os.path.basename(test_list[test_num]))
        file_name = ''.join(file_name) + ' (' + str(test_num + 1) + '/' + str(len(test_list)) + ')'
        name_label.configure(text=file_name)
    
    
#7. 사진 업로드 함수
def Draw_img(image_path):
    uploaded = Image.open(image_path)  # os에 있는 이미지를 불러옵니다.
    # uploaded.thumbnail(((top.winfo_width()/2.25), (top.winfo_height()/2.25)))  # 사이즈를 조절하고
    uploaded.thumbnail(((800/1.9), (600/1.4)))  # 사이즈를 조절하고
    img = ImageTk.PhotoImage(uploaded)  # 사진을 img 에 담습니다.
    
    sign_image.configure(image=img)  # 불러온 사진을 sign_image 라는 변수로 구성
    sign_image.image = img           # 불러온 사진을 sign_image.image 에 담습니다.
    sign_image.pack()                # 불러온 사진을 완전히 지정하고
    sign_image.place(x=228, y=220)  # 위치 시킵니다.
    
    
#8. 확률 그래프 그리는 함수
def Draw_graph(pred):
    global plot
    global graph_flag
    
    if not graph_flag:
        fig = plt.Figure(figsize=(4.25, 0.95), facecolor='#c7c7c7')  # 그래프의 크기와 색깔을 지정
        plot = FigureCanvasTkAgg(fig, master=top)  # tkinter 위에 그래프 올리기

        y = ['human %0.3f'%pred[0][0], 'AI %0.3f'%pred[0][1]]  # 소수점이하 3번째까지만 2개의 확률 저장
        fig.add_subplot(1,1,1).barh(y, pred[0], color='#142f3a')  # 그래프 그리기, 막대 색깔 설정
        fig.tight_layout()
        
        plot.draw()                  # 그래프 그리기
        plot.get_tk_widget().pack()  # 그래프 설정을 저장하기
        plot.get_tk_widget().place(x=215, y=445)  # 그래프 위치 지정하기
        
        graph_flag = True


#9. 소리를 재생하는 함수
def Play():
    global is_off
    global MUSIC_END
    
    if test_list[test_num]:  # test_list 에 음원의 위치와 이름이 들어있다면
        if is_off:  # 소리가 재생 중이 아니라면
            mixer.init()  # 초기화 작업 수행
            mixer.music.load(test_list[test_num])  # 음원을 불러들임
            mixer.music.play()  # 음원 재생
            btn_play.config(image=pause)
            is_off = False
            
            # 소리가 끝났는지 감지하기 위해
            MUSIC_END = pygame.USEREVENT+1
            pygame.mixer.music.set_endevent(MUSIC_END)
            check_event()

        else:  # 소리가 재생 중이라면
            mixer.music.pause()
            btn_play.config(image=play)
            is_off = True

            
#10. 소리가 끝났는지 감지하는 함수    
def check_event():
    global is_off
    
    for event in pygame.event.get():
        if event.type == MUSIC_END:
            print('')
            btn_play.config(image=play)
            is_off = True
    top.after(100, check_event)  # 0.1초마다 이 함수를 반복
          
            
#11. 이전/다음 파일 가져오는 함수
def Next(flag):
    global test_num
    global is_off
    global main_img
    global graph_flag
    fin = False
    
    if not test_list:  # 선택한 파일이 없다면
        tk.messagebox.showwarning("", "파일을 선택해주세요.")
        
    else:
        if flag == 'plus':  # 다음 버튼
            test_num += 1
            if test_num >= len(test_list):
                tk.messagebox.showwarning("", "마지막 파일입니다.")
                test_num -= 1
                fin = True

        elif flag == 'minus':  # 이전 버튼
            test_num -= 1
            if test_num < 0:
                tk.messagebox.showwarning("", "첫번째 파일입니다.")
                test_num += 1
                fin = True

        if not fin:  # 이전/다음 버튼으로 파일을 변경했을 때
            Draw_img(main_img)
            label_fname(test_num)
            label.configure(text='')
            if graph_flag:  # 확률 그래프가 그려져 있다면
                plot.get_tk_widget().destroy()
                graph_flag = False

        # 재생중인 소리 중단
        mixer.music.stop()
        mixer.music.unload()
        btn_play.config(image=play)
        is_off = True



##########4. 필요한 버튼 

#업로드 버튼 구현
upload_btn = Button(top, text="Upload an sound", command=upload_sound , width=18,height=2,relief='ridge')
upload_btn.configure(background="#393939", foreground='#ffffff', font=('arial',11, "bold"))  # 버튼 색깔
upload_btn.place(x=12,y=470)

# 분류 버튼 구현 
classify_btn = Button(top, text="classify an sound", command=classify, width=18,height=2,relief='ridge')
classify_btn.configure(background="#393939", foreground='#ffffff', font=('arial',11, "bold"))  # 버튼 색깔
classify_btn.place(x=12,y=525)

# 이전 버튼 구현
before = PhotoImage(file="C:\\data\\btn_img\\skip-back-fill.png")
btn_before=Button(top, image=before, width=33, height=33, background="#8f8f8f", command=lambda:Next('minus'))
btn_before.place(x=25, y=600)

# 소리 재생 버튼 구현
play = PhotoImage(file="C:\\data\\btn_img\\play-fill.png")
pause = PhotoImage(file="C:\\data\\btn_img\\pause-fill.png")
btn_play=Button(top, image=play, width=33, height=33, background="#8f8f8f", command=Play)
btn_play.place(x=79, y=600)

# 다음 버튼 구현
next = PhotoImage(file="C:\\data\\btn_img\\skip-forward-fill.png")
btn_next=Button(top, image=next, width=33, height=33, background="#8f8f8f", command=lambda:Next('plus'))
btn_next.place(x=134, y=600)

# 화면에 출력할 결과 구현 
label = Label(top, foreground="#020b18", font=('arial',100,'bold'))
label.pack(side=BOTTOM, expand=True)

# 사진 붙이기
sign_image = Label(top, bd=0)
sign_image.pack(side=BOTTOM, expand=True)
sign_image.place(x=223, y=180)
Draw_img(main_img)  # 초기 화면 이미지 부착

# 파일 이름 출력
name_label = Label(background='#393939', foreground='#ffffff', font=('arial',12,'bold'))
name_label.pack(side=BOTTOM, expand=True)
name_label.place(x=19, y=430)

# 저작권 표시
copy = Label(top, text="Copyright 2022.RVR Corp. All rights reserved.")  # compound 는 text 자리 지정
copy.configure(background="#c7c7c7", foreground='black', font=('Courier',8,'normal'))  
copy.pack(side=BOTTOM, anchor = 's', expand=False)
copy.place(x=208, y=662)
copy = Label(top, text="Send email to realvoice_RNN@gmail.com for use.")  # compound 는 text 자리 지정
copy.configure(background="#c7c7c7", foreground='black', font=('Courier',8,'normal'))  
copy.pack(side=BOTTOM, anchor = 's', expand=False)
copy.place(x=208, y=678)



pygame.init()
top.mainloop() # 종료 될 때까지 실행